### Adversarial Patch Creation
Developed by Tal Erez
[![Open In Collab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/AIPI-590-XAI/Duke-AI-XAI/blob/dev/adversarial-ai-example-notebooks/in_class_adversarial_activity.ipynb)

In [ ]:
!pip install torch torchvision adversarial-robustness-toolbox matplotlib


import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from art.attacks.evasion import FastGradientMethod
from art.estimators.classification import PyTorchClassifier